In [1]:
import os
import pandas as pd
import numpy as np
from importlib import reload

import data_wrangle
import spectral
import analysis
import utils

from prettytable import PrettyTable as pt

#From PCA
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.model_selection import cross_val_score

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import scipy.signal as ss
import pybaselines.spline as py

from math import ceil
from collections import Counter

# import plotly.io as pio
# pio.renderers.default = "browser"
# pio.renderers.default = "png"

# Initalize Object and set constants

In [2]:
path='/mnt/c/Users/16162/Desktop/dopamine.db'
table='arsenic'

## Look at the Table

In [3]:
reload(data_wrangle)
db=data_wrangle.DataTable(path,table)
db.summary(detail='low')
db.__del__()

All IDs: 225
Selected IDs: 225
Condition: None


# Grab data
---

In [4]:
reload(utils)
reload(data_wrangle)
reload(analysis)

#Initialize DataTable instance
db=data_wrangle.DataTable(path, table)

#Set condition

#Get raw data
rawdata=db.dataset(summary=False)

#Make dictionaries for naming and grouping
concen=db.label_dict('As3','As5', include_col_name=True)
soln=db.label_dict('solution', include_col_name=False)
groups=utils.invert_dict(concen)

#Close connection to database
db.__del__()

#Preprocess data
data=data_wrangle.DataSet(rawdata,snv=False, trunc_before=700).getdf()

#***test trunc_after****

# Spectra

In [5]:
myorder=['0 As3, 0 As3',
         '0 As3, 150 uM As5',
 '0 As3, 1 mM As5',
 '19 uM As3, 1 mM As5',
 '50 uM As3, 150 uM As5',
 '19 uM As3, 150 uM As5',
 '50 uM As3, 1 mM As5',
 '19 uM As3, 0 As5',
 '50 uM As3, 0 As5']



mysets={'Arsenite (III)' : ['0 As3, 150 uM As5','0 As3, 1 mM As5',],
        'Mixture':['19 uM As3, 0 As5','50 uM As3, 0 As5'],
        'Arsenate (V)':['19 uM As3, 1 mM As5','50 uM As3, 150 uM As5','19 uM As3, 150 uM As5','50 uM As3, 1 mM As5',],
        'Blank' :['0 As3, 0 As3']
        }

In [21]:
reload(spectral)
ave_dict=spectral.Spectral(data, averaging_dict=groups).get()
# obj=spectral.View(ave_dict, data.columns.tolist())

spectral.Traces(ave_dict, data.columns.tolist(), selected_groups=myorder, set_dict=mysets).set_subplots()


# View spectra

In [ ]:
#3) Make color dictionary
for item in set(namedic.values()):
    print(f'\'{item}\':\'#\',',)

#4) Make set dictionary

In [ ]:
color_dic={
'19 uM As3, 1 mM As5':'#8200ff',
'0 As3, 150 uM As5':'#6600ff',
'0 As3, 1 mM As5':'#250094',
'19 uM As3, 0 As5':'#ff0061',
'0 As3, 0 As5':'grey',
'19 uM As3, 150 uM As5':'#da00c6',
'50 uM As3, 0 As5':'#b40000',
'50 uM As3, 1 mM As5':'#a800e7',
'50 uM As3, 150 uM As5':'#ff00ae',
}

color_dic={
'0 As3, 1 mM As5':'#250094',
'0 As3, 150 uM As5':'#00eaff',
'50 uM As3, 150 uM As5':'#824c96',
'19 uM As3, 150 uM As5':'#433466',
'50 uM As3, 1 mM As5':'#ff7a00',
'19 uM As3, 1 mM As5':'#ed733f',
'19 uM As3, 0 As5':'#ff7a00',
'50 uM As3, 0 As5':'#ff0000',
'0 As3, 0 As5':'grey',}

color_dic={
'0 As3, 1 mM As5':'#2abb1d',
'0 As3, 150 uM As5':'#4ded30',

'50 uM As3, 150 uM As5':'#d1022f',
'19 uM As3, 150 uM As5':'#9f0261',
'50 uM As3, 1 mM As5':'#7c0284',
'19 uM As3, 1 mM As5':'#5102af',

'19 uM As3, 0 As5':'#e05a00',
'50 uM As3, 0 As5':'#a34100',

'0 As3, 0 As5':'grey',}

set_dic={
  'Mix':['50 uM As3, 1 mM As5', '19 uM As3, 1 mM As5', '19 uM As3, 150 uM As5', '50 uM As3, 150 uM As5'],
  'As3':['50 uM As3, 0 As5','19 uM As3, 0 As5'],
  'As5':['0 As3, 1 mM As5','0 As3, 150 uM As5'],
  'Blank':['0 As3, 0 As5']
 }

# PCA

In [178]:
reload(analysis)
var=analysis.multivar(data,data.columns,date,loc,ncomp=200)

# pc,a,b=var.pca_run()
# var.pca_fig('PC 1', 'PC 2')
# var.pca_fig('PC 1', 'PC 3')
# # var.pca_fig('PC 2', 'PC 3')
# var.pca_3d().show()
var.loo(mute=False)




Currently performing LOO on sample #0
Currently performing LOO on sample #1
Currently performing LOO on sample #2
Currently performing LOO on sample #3
Currently performing LOO on sample #4
Currently performing LOO on sample #5
Currently performing LOO on sample #6
Currently performing LOO on sample #7
Currently performing LOO on sample #8
Currently performing LOO on sample #9
Currently performing LOO on sample #10
Currently performing LOO on sample #11
Currently performing LOO on sample #12
Currently performing LOO on sample #13
Currently performing LOO on sample #14
Currently performing LOO on sample #15
Currently performing LOO on sample #16
Currently performing LOO on sample #17
Currently performing LOO on sample #18
Currently performing LOO on sample #19
Currently performing LOO on sample #20
Currently performing LOO on sample #21
Currently performing LOO on sample #22
Currently performing LOO on sample #23
Currently performing LOO on sample #24
Currently performing LOO on sample 

'Average MSE: 0.0243\nAverage RMSE: 0.1559'

In [ ]:
var.loo()

In [ ]:
var.pca_fig('PC 1', 'PC 2')

In [ ]:
ls=var.loadings('1','2', '3')
for l in ls:
    l.show()


# Extras!

## Update Database Find and Replace

In [ ]:
reload(data_wrangle)
db=data_wrangle.DataTable('/mnt/c/Users/16162/Desktop/Brain.db','sheep')
db.condition='WHERE sheep IS 2'
db.data_info('nanotag') #Add 'condition' to find_replace
# db.find_replace('BoooTh', 'Booo', 'nanotag')

## Making Figure - old code

In [ ]:
reload(utils)
#This should go into spectral
def truncate(x_data, y_data, before_target=None, after_target=None):
    '''targets need to be intergers for the Raman Shift values.
    x_data = Raman Shifts as a series or list
    y_data = df where each row is a sample
    If no only 1 target is given, this should still work'''#***
    #Find indexes for truncating
    if before_target==None:
        before_idx=0
    else:
        before_idx=utils.find_index(x_data, before_target)
    if after_target==None:
        after_idx=len(x_data)
    else:
        after_idx=utils.find_index(x_data, after_target)
    print(after_idx,'FIX return error if NONE')
    res_x=x_data[before_idx:after_idx]
    res_ys=y_data.iloc[:,before_idx:after_idx]
    t1=go.Scatter(x=x_data, y=y_data.iloc[1], name='before')
    t2=go.Scatter(x=res_x, y=res_ys.iloc[1], name='after')
    traces=[t1,t2]
    fig=go.Figure(traces)
    fig.update_layout(title='Visualize Data Truncation')
    fig.show()

    return res_x, res_ys

# x,y=truncate(x,ys,500,1080)

## Find and remove NaN values

In [ ]:
#Find NaN values
nan_cols=ys.isna().any() #Find columns with NaN values
nan_rows=ys.loc[:,nan_cols].isna().any(axis=1) #Find the rws with NaN values (in columns with NaN values)
nan_rows.index #Get indexes of rows (= sample ID)

#Drop rows with more than 5 NaN values
ys=ys.dropna(thresh=ys.shape[1]-5)

#Map namedic to sample ID
tag=ys.index.map(lambda x: namedic[x][0])

## Something to do with dataframe manipulation

In [ ]:
df=pd.DataFrame([[1,2,3,4,5],[2,3,4,5,6],[1,2,3,4,5],[5,6,7,8,9],[1,2,3,4,5],[5,6,7,8,9]])
print(df)
uniques=set(map(tuple,df.values))
uniques=list(uniques)
for u in uniques:
    mask=np.all(df==u, axis=1)
    print(mask)
print(df[mask])
print(df)
